In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('titanic_lr').getOrCreate()

In [3]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
cols = df.select(
['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked']
)

In [7]:
cols.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     3|  male|null|    0|    0| 8.4583|       Q|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       

In [8]:
final_data = cols.na.drop()

In [9]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [14]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

In [15]:
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [16]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [17]:
assembler = VectorAssembler(
    inputCols=['Pclass', 'SexVec', 'Age','SibSp', 'Parch', 'Fare','EmbarkVec'],
    outputCol='features'
)

In [18]:
from pyspark.ml.classification import LogisticRegression

In [19]:
from pyspark.ml import Pipeline

In [20]:
lr = LogisticRegression(featuresCol='features', labelCol='Survived')

In [22]:
pipeline = Pipeline(
    stages=[
        gender_indexer, embark_indexer,
        gender_encoder, embark_encoder,
        assembler, lr
    ]
)

In [23]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [24]:
fit_model = pipeline.fit(train_data)

In [25]:
results = fit_model.transform(test_data)

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [27]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [28]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [29]:
AUC = my_eval.evaluate(results)

In [30]:
AUC

0.8365357839042049